# HW5 : Math for Robotics

Author: Ruffin White  
Course: CSE291  
Date: Mar 23 2018

In [ ]:
# Make plots inline
%matplotlib inline

# Make inline plots vector graphics instead of raster graphics
from IPython.display import set_matplotlib_formats
# set_matplotlib_formats('pdf', 'svg')
set_matplotlib_formats('png', 'pdf')
# set_matplotlib_formats('png')

# import modules for plotting and data analysis
import matplotlib.pyplot as plt
from IPython.display import HTML
from log_progress import log_progress

## 1. 

> The world model is shown in figure 1. The robot is a differential drive
system of size 50x50.

1. Generate the configuration space for the robot with a grid size of 2x2 and 5 deg in angular resolution. Generate an illustration of what the configuration space looks like with the robot at orientations 0, 45 and 90 deg.
2. Use greedy search find the shortest path between start-point (50,50) and end-point (750,250). Illustrate the path and provide its length.
3. Compute the safest path from start to finish (hint: medial axis transform). Illustrate the path and provide its length.
4. Use probabilistic roadmaps (PRM) to compute a path between startand end-points with 50, 100 and 500 sample points. What is the difference in path length? Illustrate each computed path.
5. Do the same with Rapid exploring random trees (RRT). What are the main differences in performance between PRM and RRT? Illustrate each path.

## Configuration Space

In [ ]:
import cv2
import numpy as np
import pandas as pd
import pathlib
import scipy.spatial
import scipy.signal

from matplotlib import animation
from collections import OrderedDict
from PIL import Image

In [ ]:
import warnings
warnings.filterwarnings('ignore')

We'll start by defining a grid-based environment to encapsulate a discretized representation of our world model. We then will populate the 2D grid environment with the set number of wall objects that constrain possible paths what limits our work space.

In [ ]:
scale = 2

width = int(800/scale)
hight = int(300/scale)
seg1 = int(200/scale)
seg2 = int(600/scale)
seg3 = int(400/scale)
seg4 = int(100/scale)

map_2d = np.zeros(shape=(width, hight))
map_2d[0,:] = 1
map_2d[:,0] = 1
map_2d[:,hight-1] = 1
map_2d[width-1,:] = 1

map_2d[seg1-1,0:seg1] = 1
map_2d[seg2-1,0:seg1] = 1

map_2d[seg3-1,hight-1-seg1:hight-1] = 1
map_2d[seg3-1-seg4:seg3-1+seg4,hight-1-seg1] = 1

Well also define our Robot as pixelated shape, subject to the same scale used in deriving the environment. 

In [ ]:
rwidth = int(50/scale)
rhight = int(50/scale)
robot_2d = np.ones(shape=(rwidth, rhight))

Here we’ll define a function to rotate our robot representation given an angle in degrees.

In [ ]:
def rotate_bound(image, angle):
    # borrowed from https://www.pyimagesearch.com/2017/01/02/rotate-images-correctly-with-opencv-and-python/
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w // 2, h // 2)
 
    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), -angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])
 
    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))
 
    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY
 
    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH))

Given both are world and robot representations I represented as occupancy and 2D arrays, we can use simple 2d convolution, optimized with the FFT  transform,  to generate our configuration space with respect to a given angle, or list of  angles In which the robot is rotated.

In [ ]:
def get_config_2d(map_2d, robot_2d_rot):
    config_2d = scipy.signal.fftconvolve(map_2d, robot_2d_rot, mode='same')
    return config_2d

def get_config_space(map_2d, robot_2d_rot, angles):
    config_space = np.empty((width, hight, angle_res))
    for i, angle in enumerate(angles):
        robot_2d_rot = rotate_bound(robot_2d, angle)
        config_space[:,:,i] = get_config_2d(map_2d, robot_2d_rot)
    return config_space

We also define a simple function to plot our workspace or partial configuration space for visualization.

In [ ]:
def plot_map_2d(map_2d, hold=False):
    f, axs = plt.subplots(1,1,figsize=(24,64))
#     axs.imshow(map_2d.T, cmap=plt.cm.Spectral_r, interpolation='none')
    axs.imshow(map_2d.T, cmap=plt.cm.nipy_spectral, interpolation='none')
    for d in ["left", "top", "bottom", "right"]:
        axs.spines[d].set_visible(False)
    if hold:
        return f
    else:
        plt.show()

Here will generate our configuration space for the angles between 0 and 90 degrees at a degree resolution of one. Because we are using scipy’s fftconvolve function that padds our signal to next nearest power of the hamming numbers for the fft function optimized for those radixs. This makes this step quite faster than compared to traditional 2D convolution of signal/kernel or workspace/robot respectively.

In [ ]:
angle_res = 90
angles = np.linspace(start=0, stop=90, num=angle_res, endpoint=False)
config_space = get_config_space(map_2d, robot_2d, angles)

Plotting this grid based world, we can see the layout as shown in the assignment figure. 

In [ ]:
plot_map_2d(map_2d)

In the next three figures, we can visualize the configuration space 4 robot rotations at 0, 45, and 90 degrees respectively. Because our configuration space was acquired using to the convolutions, we can visualize the raw response to provide additional insight in robot collisional intensity, as well as a regionalized cost map.

In [ ]:
robot_2d_rot = rotate_bound(robot_2d, 0)
config_2d = get_config_2d(map_2d, robot_2d_rot)
# config_2d = np.clip(a=config_2d, a_min=0, a_max=1)
plot_map_2d(config_2d)

In [ ]:
robot_2d_rot = rotate_bound(robot_2d, 45)
plt.imshow(robot_2d_rot)

In [ ]:
# robot_2d_rot = rotate_bound(robot_2d, 45)
config_2d = get_config_2d(map_2d, robot_2d_rot)
# config_2d = np.clip(a=config_2d, a_min=0, a_max=1)
plot_map_2d(config_2d)

In [ ]:
robot_2d_rot = rotate_bound(robot_2d, 90)
config_2d = get_config_2d(map_2d, robot_2d_rot)
# config_2d = np.clip(a=config_2d, a_min=0, a_max=1)
plot_map_2d(config_2d)

Because our robot is rotationally symmetrical across every half pi radians, we can flatten the configuration space across rotations from 0 to 90 degrees by taking the max value along the 3rd axis on our configuration space 3D array representation. This renders the most conservative configuration space where the black region denote all points in the world reference frame that our robot can freely rotate in place.

This is useful for robotic navigation stacks that commonly make use of rotational recovery behaviors when losing localization or determining alternate routes. The astute reader will notice that this is the same result as in taking the  the 2D deconvolution of the world map  with respect to a kernel that is a  simple Circle with a radius defined as the maximum distance a collision point along the robot geometry is with respect to the center of its rotation (assuming our differential drive model).

In [ ]:
config_space_2d = config_space.max(axis=2)
# config_space_2d = np.clip(a=config_space_2d, a_min=0, a_max=0.001)
plot_map_2d(config_space_2d)

In [ ]:
config_space_2d_bool = config_space.max(axis=2)

thresh = 0.001
config_space_2d_bool[config_space_2d < thresh] = 0
config_space_2d_bool[config_space_2d >= thresh] = 1
plot_map_2d(config_space_2d_bool)

We can define a function to sweep through the configuration space in an animated fashion. However this is quite slow using to matplotlib.

In [ ]:
import matplotlib
# matplotlib.use('GTKAgg')

def plot_3d(config_space, animated=False, stride=1):
#     config_space = config_space[::stride]
    
    fig, axs = plt.subplots(1,1,figsize=(16,6))
    fig.canvas.draw()
    

    def init():
        axs.imshow(config_space[:,:,0].T, cmap=plt.cm.Spectral_r, interpolation='none')
        plt.axis('off')
        plt.tight_layout()
#         for d in ["left", "top", "bottom", "right"]:
#             axs.spines[d].set_visible(False)
        
        return fig,
    
    def animate(i):
        axs.imshow(config_space[:,:,i].T, cmap=plt.cm.Spectral_r, interpolation='none')
        return fig,
    
    if animated:
        frames = log_progress(range(angle_res), 1)
        anim = animation.FuncAnimation(fig, animate, init_func=init,
                                       frames=frames, interval=20, blit=True)
        video = anim.to_html5_video()
        return video
    else:
        init()
        return fig

# empty2_video = plot_3d(config_space, animated=True)
# display(HTML(empty2_video))

A better alternative is perhaps using pygtgraph and let OpenGL do the heavy lifting for us in rendering the frames.

In [ ]:
# import numpy as np
# from pyqtgraph.Qt import QtCore, QtGui
# import pyqtgraph as pg

# # Interpret image data as row-major instead of col-major
# pg.setConfigOptions(imageAxisOrder='row-major')

# app = QtGui.QApplication([])

# ## Create window with ImageView widget
# win = QtGui.QMainWindow()
# win.resize(800,300)
# imv = pg.ImageView()
# win.setCentralWidget(imv)
# win.show()
# win.setWindowTitle('pyqtgraph example: ImageView')

# data = config_space[:,:,:]
# # data = np.clip(a=data, a_min=0, a_max=1)


# ## Display the data and assign each frame a time value from 1.0 to 3.0
# imv.setImage(data.T, xvals=angles)

# # Get the colormap
# colormap = plt.cm.nipy_spectral  # cm.get_cmap("CMRmap")
# colormap._init()
# lut = (colormap._lut * 255).view(np.ndarray)  # Convert matplotlib colormap from 0-1 to 0 -255 for Qt
# imv.imageItem.setLookupTable(lut)

# ptr = 0
# def update():
#     global data, ptr
#     imv.imageItem.updateImage(data[:,:,ptr].T)
#     imv.imageItem.setLookupTable(lut)
#     ptr += 1
#     ptr = ptr % angles.size
# timer = QtCore.QTimer()
# timer.timeout.connect(update)

# ## Start Qt event loop unless running in interactive mode.
# if __name__ == '__main__':
#     import sys
#     if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
#         timer.start(50)
#         QtGui.QApplication.instance().exec_()


We can also use pygtgraph to slice the Configuration space arbitrarily, similarly done with MRI scans.

In [ ]:
# import numpy as np
# from pyqtgraph.Qt import QtCore, QtGui
# import pyqtgraph as pg

# app = QtGui.QApplication([])

# ## Create window with two ImageView widgets
# win = QtGui.QMainWindow()
# win.resize(800,800)
# win.setWindowTitle('pyqtgraph example: DataSlicing')
# cw = QtGui.QWidget()
# win.setCentralWidget(cw)
# l = QtGui.QGridLayout()
# cw.setLayout(l)
# imv1 = pg.ImageView()
# imv2 = pg.ImageView()
# l.addWidget(imv1, 0, 0)
# l.addWidget(imv2, 1, 0)
# win.show()

# roi = pg.LineSegmentROI([[10, 64], [120,64]], pen='r')
# imv1.addItem(roi)

# x1 = np.linspace(-30, 10, 128)[:, np.newaxis, np.newaxis]
# x2 = np.linspace(-20, 20, 128)[:, np.newaxis, np.newaxis]
# y = np.linspace(-30, 10, 128)[np.newaxis, :, np.newaxis]
# z = np.linspace(-20, 20, 128)[np.newaxis, np.newaxis, :]
# d1 = np.sqrt(x1**2 + y**2 + z**2)
# d2 = 2*np.sqrt(x1[::-1]**2 + y**2 + z**2)
# d3 = 4*np.sqrt(x2**2 + y[:,::-1]**2 + z**2)
# # data = (np.sin(d1) / d1**2) + (np.sin(d2) / d2**2) + (np.sin(d3) / d3**2)
# data = config_space[:,:,:]

# def update():
#     global data, imv1, imv2
#     d2 = roi.getArrayRegion(data, imv1.imageItem, axes=(1,2))
#     imv2.setImage(d2)
    
# roi.sigRegionChanged.connect(update)


# ## Display the data
# imv1.setImage(data)
# imv1.setHistogramRange(-0.01, 0.01)
# imv1.setLevels(-0.003, 0.003)

# update()

# ## Start Qt event loop unless running in interactive mode.
# if __name__ == '__main__':
#     import sys
#     if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
#         QtGui.QApplication.instance().exec_()

Lastly we can export the occupancy grid from our configuration workspace and to a three dimensional Point cloud. Obviously this only works because of the 2D dimension of our initial world frame with the additional dimension of rotation. However the same slicing technique, as above, could be used to introspect the configuration space for higher dimensions.

In [ ]:
# from plyfile import PlyData, PlyElement
# data = np.clip(a=config_space, a_min=0, a_max=1)
# points = np.asarray(np.where(data == 1)).T
# vertex = np.array(list(map(tuple, points)), dtype=[('x', 'u2'), ('y', 'u2'), ('z', 'u2')])
# el = PlyElement.describe(vertex, 'config_space')
# PlyData([el]).write('config_space.ply')

In [ ]:
# # import numpy as np
# import yaml
# matrix = np.random.randint(2, size=(10,7))
# with open('foo.yaml', 'w') as f:
#     yaml.dump(config_space.flatten().tolist(), f)

In [ ]:
# thresh = 0.001
# config_space_bool = config_space.copy()
# config_space_bool[config_space < thresh] = 0
# config_space_bool[config_space >= thresh] = 1
# plot_map_2d(config_space_bool[:,:,45])

In [ ]:
# np.savetxt('config_space_bool.yaml', config_space_bool.flatten(), delimiter=',', fmt='%d,')

## Greedy Search

In [ ]:
import oyaml as yaml
import collections

def search2yaml(start, goal, map_grid, path):
    meta = []
    meta.append({'start': start[::-1].tolist()})
    meta.append({'goal': goal[::-1].tolist()})
    meta.append({'origin': np.zeros(len(map_grid.shape), dtype=np.int).tolist()})
    meta.append({'dim': list(map_grid.shape[::-1])})
    meta.append({'resolution': 1})
    meta.append({'data': map_grid.flatten().astype(np.int).tolist()})
    with open(path, 'w') as outfile:
        yaml.dump(list(meta), outfile)

In [ ]:
start = (np.array([50,50])/scale).astype(np.int)
goal = (np.array([750,250])/scale).astype(np.int)

config_space_2d_bool_path = '/tmp/config_space_2d_bool.yaml'
search2yaml(start=start, goal=goal, map_grid=config_space_2d_bool, path=config_space_2d_bool_path)

In [ ]:
jps_build_path = '/home/ruffsl/git/sikang/jps3d/build'
output_dir = '/tmp/'

In [ ]:
%%script env JPS_BUILD_PATH="$jps_build_path" YAML_FILE="$config_space_2d_bool_path" OUTPUT_DIR="$output_dir" bash

cd $JPS_BUILD_PATH
./test_planner_2d $YAML_FILE $OUTPUT_DIR

In [ ]:
import os

def read_solved_paths(output_dir):
    path_jps_file = os.path.join(output_dir, 'path_jps.csv')
    path_astar_file = os.path.join(output_dir, 'path_astar.csv')
    path_jps = np.genfromtxt(path_jps_file, delimiter=',', dtype=np.int).T
    path_astar = np.genfromtxt(path_astar_file, delimiter=',', dtype=np.int)[::-1].T
    path_jps[[0, 1],:] = path_jps[[1, 0],:]
    path_astar[[0, 1],:] = path_astar[[1, 0],:]
    return path_jps, path_astar

In [ ]:
path_jps, path_astar = read_solved_paths(output_dir)

fig = plot_map_2d(config_space_2d_bool, hold=True)
ax = fig.get_axes()[0]

ax.plot(path_astar[0], path_astar[1], label="A* Search")
ax.plot(path_jps[0], path_jps[1], '--', label="Jump Point Search")
plt.legend()
plt.show()

In [ ]:
a, b = int(rwidth/2-1), int(rwidth/2-1)
n = rwidth
r = int(rwidth/2)
y,x = np.ogrid[-a:n-a, -b:n-b]
mask = x*x + y*y <= r*r

optimistic_robot_2d = np.zeros((n, n))
optimistic_robot_2d[mask] = 1

In [ ]:
plt.imshow(optimistic_robot_2d)
plt.show()

In [ ]:
optimistic_config_space_2d = get_config_2d(map_2d, optimistic_robot_2d)
plot_map_2d(optimistic_config_space_2d)


optimistic_config_space_2d_bool = optimistic_config_space_2d.copy()
optimistic_config_space_2d_bool[optimistic_config_space_2d < thresh] = 0
optimistic_config_space_2d_bool[optimistic_config_space_2d >= thresh] = 1
# plot_map_2d(optimistic_config_space_2d_bool)

In [ ]:
optimistic_config_space_2d_bool_path = '/tmp/optimistic_config_space_2d_bool.yaml'
search2yaml(start=start, goal=goal, map_grid=optimistic_config_space_2d_bool, path=optimistic_config_space_2d_bool_path)

In [ ]:
%%script env JPS_BUILD_PATH="$jps_build_path" YAML_FILE="$optimistic_config_space_2d_bool_path" OUTPUT_DIR="$output_dir" bash

cd $JPS_BUILD_PATH
./test_planner_2d $YAML_FILE $OUTPUT_DIR

In [ ]:
path_jps, path_astar = read_solved_paths(output_dir)

fig = plot_map_2d(optimistic_config_space_2d_bool, hold=True)
ax = fig.get_axes()[0]

ax.plot(path_astar[0], path_astar[1], label="A* Search")
ax.plot(path_jps[0], path_jps[1], '--', label="Jump Point Search")
plt.legend()
plt.show()

Because of the Triangle inequality, we know the paths generated from our Manhattan based configuration space will always be of a longer distance than that of pure continuous euclidean solutions.

## Safest Path

Computing the safest is path through our workspace environment will in this case equate as being the trajectory that is  maximally distant from all nearby obstacles. Essentially this necessitates computing the voronoi partitions for the empty regions in the work space. We can then travers the set of points generated from this tessellation as it provides us a simplistic roadmap. We then need only find the closest decomposed points to both the start and the goal. As a final check, we can sample the resulting trajectory using our configuration space as a collisions look up.

In [ ]:
points = np.asarray(np.where(map_2d == 1)).T
vor = scipy.spatial.Voronoi(points)

vertices = vor.vertices[::1].T.astype(int)
map_2d_vert = map_2d.copy()
map_2d_vert[vertices[0],vertices[1]] = 2
map_2d_vert[map_2d == 1] = 1

plot_map_2d(map_2d_vert)

In [ ]:
map_2d_vert_bool = map_2d_vert.copy()
map_2d_vert_bool[map_2d_vert == 2] = 0
map_2d_vert_bool[map_2d_vert != 2] = 1

map_2d_vert_bool_path = '/tmp/map_2d_vert_bool.yaml'
search2yaml(start=start, goal=goal, map_grid=map_2d_vert_bool, path=map_2d_vert_bool_path)

In [ ]:
%%script env JPS_BUILD_PATH="$jps_build_path" YAML_FILE="$map_2d_vert_bool_path" OUTPUT_DIR="$output_dir" bash

cd $JPS_BUILD_PATH
./test_planner_2d $YAML_FILE $OUTPUT_DIR

In [ ]:
path_jps, path_astar = read_solved_paths(output_dir)

fig = plot_map_2d(map_2d, hold=True)
ax = fig.get_axes()[0]

ax.plot(path_astar[0], path_astar[1], label="A* Search")
ax.plot(path_jps[0], path_jps[1], '--', label="Jump Point Search")
plt.legend()
plt.show()

## Probabilistic Road Maps

In [ ]:
def sample_points(map_2d, n_sample):
    map_2d_flat = map_2d.flatten()
    
    map_2d_p = ((map_2d.flatten()-1)*-1).astype(np.int)
    map_2d_p = map_2d_p/map_2d_p.sum()
    map_2d_ix = np.arange(0, map_2d_flat.size)

    samples = np.random.choice(map_2d_ix, n_sample, replace=False, p=map_2d_p)
    samples = np.array(np.unravel_index(samples, map_2d.shape))

    return samples

In [ ]:
n_samples = 50
samples = sample_points(config_space_2d_bool, n_samples)

fig = plot_map_2d(config_space_2d_bool, hold=True)
ax = fig.get_axes()[0]

ax.plot(samples[0], samples[1], 'o', label="PRM")
plt.legend()
plt.show()

In [ ]:
from skimage import measure
import networkx as nx
import heapq

In [ ]:
def norm_by_min_max(array):
    array = array - array.min()
    array = array/array.max()
    array = array/array.sum()
    return array

def sample_edges(G, n_sample, attribute='distance'):
    edgelist, edge_distances = zip(*nx.get_edge_attributes(G,attribute).items())
    edges_p = np.array(edge_distances)
    edges_p = norm_by_min_max(edges_p)
    edges_ix = np.arange(0, len(edgelist))

    samples = np.random.choice(edges_ix, n_sample, replace=False, p=edges_p)
    return samples

def sample_edges_sorted(G, n_sample, attribute='distance'):
    edgelist, edge_distances = zip(*nx.get_edge_attributes(G,attribute).items())
    edge_distances = np.array(edge_distances)
    edges_ix = np.arange(0, len(edgelist))
    
    samples = heapq.nlargest(n_sample, edges_ix, edge_distances.take)
    return samples

def prune_graph_edges(G, n_prune, prune_method):
    samples = prune_method(G, n_prune)
    edges = np.array(G.edges())
    edges = edges[samples]
    for edge in edges:
        u, v = tuple(edge[0]), tuple(edge[1])
        G.remove_edge(u, v)
    return G

In [ ]:
def distance(u, v):
    return np.linalg.norm(np.array(u)-np.array(v))

def build_prm_graph(map_2d, samples, n_prune, prune_method=sample_edges):
    G = nx.complete_graph(samples.shape[1])
    print("G.edges: ", len(G.edges()))
    
    keys = tuple(map(tuple, samples.T))
    mapping = dict(zip(G.nodes, keys))
    G = nx.relabel_nodes(G, mapping)
    
    print("distance")
    for u, v, data in G.edges(data=True):
        data['color'] = 'blue'
        data['distance'] = distance(u, v)
    print("prune_graph_edges")
    G = prune_graph_edges(G, n_prune, prune_method)
    
    
    G_temp = G.copy()
    
    G_edges = G_temp.edges()
    print("G_edges: ", len(G_edges))
    for edge in G_edges:
        trajectory = measure.profile_line(map_2d, edge[0], edge[1])
        if trajectory.any():
            G.remove_edge(*edge)

    for node, data in G.nodes(data=True):
        data['color'] = 'purple'
        data['pos'] = node
    return G

In [ ]:
ratio_prune = 0
# ratio_prune = 0.98
n_prune = int((n_samples*(n_samples-1)/2)*ratio_prune)
G = build_prm_graph(config_space_2d_bool, samples, n_prune)

n(n − 1)/2

In [ ]:
fig = plot_map_2d(config_space_2d_bool, hold=True)
ax = fig.get_axes()[0]

pos = nx.get_node_attributes(G,'pos')
edgelist, edge_color = zip(*nx.get_edge_attributes(G,'color').items())
nodelist, node_color = zip(*nx.get_node_attributes(G,'color').items())

nx.draw(G, pos=pos, ax=ax, edgelist=edgelist, edge_color=edge_color, node_color=node_color)
plt.show()

In [ ]:
from scipy import spatial

In [ ]:
def find_closest_open_path(point, tree, map_2d, G):
    dists, indexs = tree.query(x=point, k=len(G.nodes()))
    for index in indexs:
        query = tree.data[index]
        trajectory = measure.profile_line(map_2d, point, query)
        if not trajectory.any():
            return query
    raise ValueError("No open path found")

def query_prm(start, goal, map_2d, G):
    start = tuple(start)
    goal = tuple(goal)
    G = G.copy()
    nodes = G.nodes()
    tree = spatial.KDTree(nodes)
    graph_start = tuple(find_closest_open_path(start, tree, map_2d, G))
    graph_goal = tuple(find_closest_open_path(goal, tree, map_2d, G))
    G.add_node(start, pos=start, color='green')
    G.add_node(goal, pos=goal, color='red')
    G.add_edge(start, graph_start, distance=distance(start, graph_start))
    G.add_edge(goal, graph_goal, distance=distance(goal, graph_goal))
    
    path = nx.astar_path(G, source=start, target=goal, heuristic=distance, weight='distance')
    G_path = G.subgraph(path)
    for u, v, data in G_path.edges(data=True):
        data['color'] = 'orange'
    for node, data in G_path.nodes(data=True):
        if node not in [start, goal]:
            data['color'] = 'orange'
    length = nx.astar_path_length(G, source=start, target=goal, heuristic=distance, weight='distance')
    return G, path, length

In [ ]:
G, path, length = query_prm(start, goal, map_2d=config_space_2d_bool, G=G)

In [ ]:
fig = plot_map_2d(config_space_2d_bool, hold=True)
ax = fig.get_axes()[0]

pos = nx.get_node_attributes(G,'pos')
edgelist, edge_color = zip(*nx.get_edge_attributes(G,'color').items())
nodelist, node_color = zip(*nx.get_node_attributes(G,'color').items())

nx.draw(G, pos=pos, ax=ax, edgelist=edgelist, edge_color=edge_color, node_color=node_color)
plt.show()
print("PRM Length: ", length)

In [ ]:
n_samples = 100
samples = sample_points(config_space_2d_bool, n_samples)

ratio_prune = 0
ratio_prune = 0.50
n_prune = int((n_samples*(n_samples-1)/2)*ratio_prune)
G = build_prm_graph(config_space_2d_bool, samples, n_prune)

G, path, length = query_prm(start, goal, map_2d=config_space_2d_bool, G=G)

In [ ]:
fig = plot_map_2d(config_space_2d_bool, hold=True)
ax = fig.get_axes()[0]

pos = nx.get_node_attributes(G,'pos')
edgelist, edge_color = zip(*nx.get_edge_attributes(G,'color').items())
nodelist, node_color = zip(*nx.get_node_attributes(G,'color').items())

nx.draw(G, pos=pos, ax=ax, edgelist=edgelist, edge_color=edge_color, node_color=node_color)
plt.show()
print("PRM Length: ", length)

In [ ]:
n_samples = 500
samples = sample_points(config_space_2d_bool, n_samples)

ratio_prune = 0.98
n_prune = int((n_samples*(n_samples-1)/2)*ratio_prune)
G = build_prm_graph(config_space_2d_bool, samples, n_prune)

G, path, length = query_prm(start, goal, map_2d=config_space_2d_bool, G=G)

In [ ]:
fig = plot_map_2d(config_space_2d_bool, hold=True)
ax = fig.get_axes()[0]

pos = nx.get_node_attributes(G,'pos')
edgelist, edge_color = zip(*nx.get_edge_attributes(G,'color').items())
nodelist, node_color = zip(*nx.get_node_attributes(G,'color').items())

nx.draw(G, pos=pos, ax=ax, edgelist=edgelist, edge_color=edge_color, node_color=node_color)
plt.show()
print("PRM Length: ", length)

In [ ]:
n_samples = 500
samples = sample_points(config_space_2d_bool, n_samples)

ratio_prune = 0.95
n_prune = int((n_samples*(n_samples-1)/2)*ratio_prune)
G = build_prm_graph(config_space_2d_bool, samples, n_prune, prune_method=sample_edges_sorted)

G, path, length = query_prm(start, goal, map_2d=config_space_2d_bool, G=G)

In [ ]:
fig = plot_map_2d(config_space_2d_bool, hold=True)
ax = fig.get_axes()[0]

pos = nx.get_node_attributes(G,'pos')
edgelist, edge_color = zip(*nx.get_edge_attributes(G,'color').items())
nodelist, node_color = zip(*nx.get_node_attributes(G,'color').items())

nx.draw(G, pos=pos, ax=ax, edgelist=edgelist, edge_color=edge_color, node_color=node_color)
plt.show()
print("PRM Length: ", length)

## Rapid Exploring Random Trees

In [ ]:
from scipy import spatial

class RRT_Goal_Connect():
    def __init__(self, start, goal, map_2d, goalBias = 0.05, rebalance=True):

        self.tree1 = nx.DiGraph()

        self.n_iters = 0
        self.n_extensions = 0

        self.start = start
        self.goal  = goal
        self.goal_found  = False
        self.map_2d = map_2d.copy()
        self.goalBias = goalBias
        self.rebalance = rebalance
        
        map_2d_flat = map_2d.flatten()
        map_2d_p = ((map_2d.flatten()-1)*-1).astype(np.int)
        self.map_2d_p = map_2d_p/map_2d_p.sum()
        self.map_2d_ix = np.arange(0, map_2d_flat.size)
        
        self.tree1.add_node(tuple(self.start), state=self.start)
    
    def nearest_neighbor(self, tree, state):
        node_so_far = None
        distance_so_far = None
        for this_node, this_data in tree.nodes(data=True):
            this_distance = self.distance(this_data['state'], state) 
            if(distance_so_far is None):
                distance_so_far = this_distance
                node_so_far = this_node
            elif(distance_so_far > this_distance):
                node_so_far = this_node
                distance_so_far = this_distance
        return (node_so_far,distance_so_far)
    
    def search(self,iters=500):
        for i in range(iters):
            self.n_iters += 1
            if (np.random.rand() <= self.goalBias):
                q_rand = self.goal
            else:
                q_rand = self.sample()
            self.extend(q_rand)
            if self.goal_found:
                print("Goal found!")
                print("Iterations taken: ", i)
                return True
        print("No goal found...")
        print("Iterations taken: ", iters)
        return False
    
    def extend(self,q_rand):
        self.n_extensions += 1
        tree = self.tree1

        q_nearest_id, q_nearest_distance  = self.nearest_neighbor(tree, q_rand)
        x_new_id = self.extend_from(tree, q_nearest_id, q_nearest_distance, q_rand)
    
    def extend_from(self, tree, q_nearest_id, q_nearest_distance, q_rand):
        q_near = tree.node[q_nearest_id]['state']
        trajectory = measure.profile_line(self.map_2d, q_near, q_rand)
        if trajectory.any():
            if trajectory[0] or trajectory[1]:
                # pointed up against a wall, just ignore it
                return
            else:
                stop_point = np.argmax(trajectory)
                delta = (q_rand - q_near)*(stop_point/len(trajectory))
                q_new = q_near + np.floor(delta).astype(np.int)
                print(self.n_extensions, " Bumbed wall: ", q_new)
        else:
            q_new = q_rand
        
        tree.add_node(tuple(q_new), state=q_new)
        q_new_distance = self.distance(q_near, q_new)
        tree.add_edge(tuple(q_near), tuple(q_new), color='blue', distance=q_new_distance)
        self.check_goal(tree, tuple(q_new))
        
    def best_solution(self, tree, state):
        goal_id = self.nearest_neighbor(tree, state)[0]
        path = nx.shortest_path(tree, source=goal_id, target=tuple(self.goal))
        length = nx.shortest_path_length(tree, source=goal_id, target=tuple(self.goal), weight='distance')
        return path, length
    
    
    def check_goal(self, tree, node_id):
        self.goal_found = np.array_equal(self.goal, tree.node[node_id]['state'])
    
    def sample(self):
        """
        sample() returns point in state space
        """
        sample_1d_ix = np.random.choice(self.map_2d_ix, 1, replace=False, p=self.map_2d_p)
        sample_nd_ix = np.array(np.unravel_index(sample_1d_ix, self.map_2d.shape))
        q_rand = sample_nd_ix.flatten()
        if self.rebalance:
            self.map_2d_p[sample_1d_ix] = 0
            self.map_2d_p = self.map_2d_p*(1/self.map_2d_p.sum())
        return q_rand
    
    def distance(self,u,v):
        """
        distance(from_node,to_point)
        """
        return spatial.distance.euclidean(u, v)

In [ ]:
rrt_goal_connect = RRT_Goal_Connect(start, goal, config_space_2d_bool, goalBias=0.05)
rrt_goal_connect.search(10000)

In [ ]:
fig = plot_map_2d(config_space_2d_bool, hold=True)
ax = fig.get_axes()[0]

G = rrt_goal_connect.tree1
G = G.to_undirected()

path, length = rrt_goal_connect.best_solution(rrt_goal_connect.tree1, start)
G_path = G.subgraph(path)
for u, v, data in G_path.edges(data=True):
    data['color'] = 'red'

pos = nx.get_node_attributes(G,'state')
edgelist, edge_color = zip(*nx.get_edge_attributes(G,'color').items())
# nodelist, node_color = zip(*nx.get_node_attributes(G,'color').items())

# nx.draw(G, pos=pos, ax=ax, edgelist=edgelist, edge_color=edge_color, node_color=node_color)
nx.draw_networkx_edges(G, pos=pos, ax=ax, edge_color=edge_color)
plt.show()
print("RRT Length: ", length)